In [1]:
import os 
import numpy
from PIL import Image
import PIL
import pdfplumber
import io
import cv2
import rembg

In [20]:
def image_printing(filename:str = "./pdf_data/katalog-KUTE-POSTAL-2022-low3.pdf", page_number = 0):
    with pdfplumber.open(filename) as pdf:
        number_pages = len(pdf.pages)
        page = pdf.pages[page_number]  
        for image in page.images:

            image_stream = image["stream"].get_data()    
            
            image_bytes = io.BytesIO(image_stream)

            
            image = Image.open(image_bytes)
            image = image.convert("L")
            image = PIL.ImageOps.invert(image)
            
            image = rembg.remove(image)
            print(image)


In [21]:
image_printing(page_number = 16)

<PIL.Image.Image image mode=RGBA size=120x112 at 0x7F7004713010>
<PIL.Image.Image image mode=RGBA size=143x133 at 0x7F6FF87AB750>
<PIL.Image.Image image mode=RGBA size=159x153 at 0x7F700009F2D0>
<PIL.Image.Image image mode=RGBA size=175x169 at 0x7F6FF1334890>
<PIL.Image.Image image mode=RGBA size=137x241 at 0x7F6FF13B19D0>
<PIL.Image.Image image mode=RGBA size=152x239 at 0x7F700009C210>
<PIL.Image.Image image mode=RGBA size=238x188 at 0x7F700009E010>
<PIL.Image.Image image mode=RGBA size=165x113 at 0x7F6FF87A8CD0>
<PIL.Image.Image image mode=RGBA size=197x135 at 0x7F700009F810>
<PIL.Image.Image image mode=RGBA size=122x112 at 0x7F7004549C10>
<PIL.Image.Image image mode=RGBA size=145x134 at 0x7F6FF0E08610>
<PIL.Image.Image image mode=RGBA size=133x112 at 0x7F7000120B10>
<PIL.Image.Image image mode=RGBA size=158x133 at 0x7F700009FD90>
<PIL.Image.Image image mode=RGBA size=134x111 at 0x7F700009F7D0>
<PIL.Image.Image image mode=RGBA size=161x132 at 0x7F6FF0E08750>
<PIL.Image.Image image mo

In [56]:
def generate_images_id_page(filename:str = "./pdf_data/katalog-KUTE-POSTAL-2022-low3.pdf", page_number = 11):
    import re
    with pdfplumber.open(filename) as pdf:
        image_properties = []
        page = pdf.pages[page_number]
        for image_dict in page.images:
            image = image_dict["stream"].get_data()
            try:
                image = Image.open(io.BytesIO(image))
            except:
                print("Couldn't convert, Skipping")
                continue
            image = image.convert("RGB")
            image = PIL.ImageOps.invert(image)
            left, right,bottom, top = image_dict["x0"],image_dict["x1"],image_dict["bottom"],image_dict["top"]
            middle_x, middle_y = (left +  right ) / 2, (top + bottom) / 2
            image_properties.append((image, (middle_x, middle_y), (left, right, bottom, top)))
        
        text_objects = []
        
        id_regex = re.compile("\d{2,}\.+")
        
        for word in page.extract_words():
            if id_regex.match(word["text"]):
                text_objects.append(word)
        
        text_properties = []
        
        for word in text_objects:
            left, right, bottom, top = word["x0"], word["x1"], word["bottom"], word["top"]
            middle_x, middle_y = (left + right) /2 , (top + bottom) / 2
            __id = word["text"]
            text_properties.append((__id, (middle_x, middle_y), (left, right, bottom, top)))
        
        matching_dict = {key[0] : [] for key in text_properties}
        
        redirect = "9999.9999"
        matching_dict[redirect] = [-1]
        for image in image_properties:
            min_dist, min_word = float("inf"), [redirect, None]
            im_x, im_y = image[1]
            _, _, im_bottom, im_top = image[2]
            for word in text_properties:
                word_x, word_y = word[1] 
                _, _, word_bottom, word_top = word[2]
                cur_distance = ((word_x - im_x) ** 2 + (word_y - im_y) ** 2) **(1/2)
                
                if word_top < im_top: continue
                if cur_distance < min_dist:
                    min_dist = cur_distance
                    min_word = word
            
            matching_dict[min_word[0]].append(image[0])
        
        #second iteration 
        
        for word in text_properties:
            output = matching_dict[word[0]]
            if len(output) != 0: 
                continue
            min_dist, min_im = float("inf"), None
            
            word_x, word_y = word[1] 
            _, _, word_bottom, word_top = word[2]
            
            for image in image_properties:
                im_x, im_y = image[1]
                _, _, im_bottom, im_top = image[2]
                cur_distance = ((word_x - im_x) ** 2 + (word_y - im_y) ** 2) **(1/2)
                if word_top < im_top: continue
                if cur_distance < min_dist:
                    min_dist = cur_distance
                    min_im = image[0]
            matching_dict[word[0]].append(min_im)
        
        
        for key, img_arr in matching_dict.items():
            
            for idx, img in enumerate(img_arr):
                name = f'{key}__{idx}.png'
                if img == -1:
                    continue
                #display(key)
                if img is None:
                    print("case")
                    continue
                
                new_background_color = (239,240,241)  # Red color, represented as (R, G, B)
                new_image = Image.new('RGB', img.size, new_background_color)
                
                img = rembg.remove(img)
                new_image.paste(img, (0, 0), img)
                
                new_image.save("result/" + name)
                

In [57]:
generate_images_id_page(page_number=11)

In [58]:
for i in range(0,220):
    try:
        generate_images_id_page(page_number=i)
    except:
        print("bad page", i)

case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
Couldn't convert, Skipping
case
case
Couldn't convert, Skipping
Couldn't convert, Skipping
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case
case

In [7]:
import cv2
from cv2 import dnn_superres
# Create an SR object - only function that differs from c++ code
sr = dnn_superres.DnnSuperResImpl_create()
# Read image
image = cv2.imread('./VaidaFilip/IMG-20230721-WA0003.jpg')
# Read the desired model
path = "FSRCNN_x3.pb"
sr.readModel(path)
# Set the desired model and scale to get correct pre- and post-processing
sr.setModel("fsrcnn", 3)
# Upscale the image
result = sr.upsample(image)
# Save the image
cv2.imwrite("./VaidaFilip/img4.png", result)
    

True